# Agent-based simulation of SIR model on random networks
Student: Luca Mihailescu (luca.mihailescu@fu-berlin.de)\
Coordinator: Sina Zendehroud, M.Sc.

## Dependencies

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy as sp
import scipy.integrate

from scipy.integrate import odeint
from scipy.stats import poisson

## Poisson graph
This is the function for generating a graph with a Poisson distribution of degrees. It uses NetworkX's `expected_degree_graph(w)` to generate a graph (i.e. network) with given expected degrees, and SciPy's `poisson.rvs(mu)` to produce the series of degrees with a given expected value.

In [ ]:
def poissongraph(n,mu):
    z= np.zeros(n) #n is number of nodes
    for i in range(n):
        z[i]=poisson.rvs(mu) #mu is the expected value
    G=nx.expected_degree_graph(z, selfloops=False)
    return G

## Analytical model computation
The SIR model:

$$\begin{equation}
    \begin{cases}
      \frac{dS}{dt}=-\frac{\beta I S}{N}\\
      \frac{dI}{dt}=\frac{\beta I S}{N}-\gamma I\\
      \frac{dR}{dt}=\gamma I
    \end{cases}\,.
\end{equation}$$

For numerically integrating this ODE, I used SciPy's `scipy.integrate.odeint(func, y0, t, args=())`. It takes as arguments the function to be integrated (in this case `model`), the initial conditions (in this case a vector of initial conditions), the sequence of time points for which to solve, and any additional arguments of the function (in this case $\beta$ and $\gamma$).

In [ ]:
#System of differential equations
def model(u, t, beta, gamma):
    S = u[0]
    I = u[1]
    R = u[2]
    N = S + I + R
    dS_dt = - beta * I * S / N
    dI_dt = beta * I * S / N - gamma * I
    dR_dt = gamma * I
    return [dS_dt, dI_dt, dR_dt]

#The upper time limit for the integration
tmax = 10.0 

#Initial conditions
S_start = 0.99
I_start = 0.01

t = np.linspace(0.0, tmax, 1000) #Time vector initialization
args = (3.2, 0.23) #Tuple of disease parameters
u0 = [S_start, I_start, 0] #Vector of initial S, I, R values to be passed to the model
y = sp.integrate.odeint(model, u0, t, args) #ODE integration

#Plot the results
fig = plt.figure(figsize=(10, 7), dpi=100)
ax = fig.add_subplot(111)
h1, = ax.plot(t, y[:,0])
h2, = ax.plot(t, y[:,1])
h3, = ax.plot(t, y[:,2])

ax.legend([h1, h2, h3], ["Susceptible", "Infected", "Removed"], fontsize=13)
ax.set_xlabel("t", fontsize=13)
ax.set_ylabel("Population", fontsize=13)
ax.text(8.9, 0.35, r'$\beta=3.2$', fontsize=13)
ax.text(8.9, 0.3, r'$\gamma=0.23$', fontsize=13)
ax.text(8.9, 0.25, r'$I_0=0.01$', fontsize=13)
ax.grid()
plt.show()

## Agent-based simulation
This is the function definition for the agent-based simulation of the SIR model on networks, as described at the seminar. It takes as arguments a graph $G$, the disease parameters $\beta$ and $\gamma$, the number of initially infected agents $I_0$, and finally the simulation step duration in days (by default 1). To run the simulation, see the section 'Run' below.

In [ ]:
def SIR_simulation(G, beta, gamma, I_0, step=1):
    
    #Setup
    N = G.number_of_nodes()
    t = 0
    
    susceptible = [i for i in range(N)]
    infected = []
    removed = []
    
    #Begin infection
    n_i0 = I_0 if I_0 != 0 else 1 
    for i in range(n_i0):
        rand = random.randrange(N)
        while not(rand in susceptible):
            rand = random.randrange(N)
        
        susceptible.remove(rand)
        infected.append(rand)
        
    n_S = [len(susceptible)]
    n_I = [len(infected)]
    n_R = [len(removed)]
        
    
    #Simulation loop
    while len(infected) != 0:
        t += step
        
        for i in infected:
            #Infection dynamics
            for j in G.neighbors(i):
                if j in susceptible and random.random() < (beta * step):
                    susceptible.remove(j)
                    infected.append(j)
        
            #Recovery dynamics
            if random.random() < (gamma * step):
                infected.remove(i)
                removed.append(i)
                
        #Record statistics
        n_S.append(len(susceptible))
        n_I.append(len(infected))
        n_R.append(len(removed))
    
    
    return t, n_S, n_I, n_R

### Visualisation

In [ ]:
def plot_evolution():
    X = t+1
    Y = n_S[0] + n_I[0] + n_R[0]

    fig = plt.figure(figsize=(10, 7), dpi=100)
    ax = fig.add_subplot(111)
    h1, = ax.plot(range(t+1), n_S)
    h2, = ax.plot(range(t+1), n_I)
    h3, = ax.plot(range(t+1), n_R)

    ax.legend([h1, h2, h3], ["Susceptible", "Infected", "Removed"], loc='upper right', fontsize=13)
    ax.set_xlabel("t", fontsize=13)
    ax.set_ylabel("Population", fontsize=13)
    ax.text(0.85*X, 0.5*Y, fr'$\beta={beta}$', fontsize=13)
    ax.text(0.85*X, 0.45*Y, fr'$\gamma={gamma}$', fontsize=13)
    ax.text(0.85*X, 0.4*Y, fr'$R_0={beta / gamma}$', fontsize=13)
    ax.text(0.85*X, 0.35*Y, fr'$I_0={I_0}$', fontsize=13)
    ax.text(0.85*X, 0.3*Y, fr'$N={N}$', fontsize=13)
    ax.text(0.85*X, 0.25*Y, fr'$param={param}$', fontsize=13)
    ax.grid()
    plt.show()

### Run

In [ ]:
beta = 0.05
gamma = 0.1
I_0 = 2
N = 10000
param = 4

#Remove comment for the desired network type 
#G = nx.erdos_renyi_graph(N, param)
#G = nx.barabasi_albert_graph(N, param)
G = poissongraph(N, param)

t, n_S, n_I, n_R = SIR_simulation(G, beta, gamma, I_0, 1)
plot_evolution()